In [ ]:
# Source for Cases Data: https://github.com/nytimes/covid-19-data.git

# Source for mobility Data: https://pastelsky.github.io/covid-19-mobility-tracker

In [1]:
import pandas as pd

import matplotlib.pyplot as plt

import requests

import json

from us_state_abbrev import us_state_abbrev

In [2]:
# Input File paths

file_us = '../covid-19-data/us.csv'

file_states = '../covid-19-data/us-states.csv'

file_counties = '../covid-19-data/us-counties.csv'

In [3]:
# Create Pandas Data Frame

us_covid_df = pd.read_csv(file_us)

us_states_df = pd.read_csv(file_states)

In [4]:
# Country Level

us_covid_df.head()

,date,cases,deaths
0,2020-01-21,1,0
1,2020-01-22,1,0
2,2020-01-23,1,0
3,2020-01-24,2,0
4,2020-01-25,3,0


In [5]:
duplicates = us_states_df['state'].tolist()

state_list = list(set(duplicates))

state_list.sort()

In [6]:
# Create a dictionary to store all the observations for each state

df = [state for _, state in us_states_df.groupby('state')]


data_frame = dict(zip(state_list, df)) 

In [7]:
# Sample to extract data for state

data_frame['Georgia'].head()

,date,state,fips,cases,deaths
256,2020-03-02,Georgia,13,2,0
271,2020-03-03,Georgia,13,2,0
287,2020-03-04,Georgia,13,2,0
305,2020-03-05,Georgia,13,2,0
326,2020-03-06,Georgia,13,3,0


In [8]:
# URL TO get mobility data for the entire country

url = "https://pastelsky.github.io/covid-19-mobility-tracker/output/US/mobility.json"

response = requests.get(url).json()

In [9]:
# Extract Mobility data for retail & recreation

locations = ['retailAndRecreation', 'parks', 'groceryAndPharmacy', 'transitStations','workplaces', 'residential']

location_df = []

for location in locations:
    
    request = response['country'][location]['points']
    
    location_df.append(request)

# retail & recreation

retail = location_df[0]

retail_df = pd.DataFrame(retail)

retail_df.rename(columns = {'value':'Per_decrease_ret_rec'}, inplace = True)


# Parks

parks = location_df[1]

parks_df = pd.DataFrame(parks)

parks_df.rename(columns = {'value':'Per_decrease_parks'}, inplace = True)


# grocery

grocery = location_df[2]

grocery_df = pd.DataFrame(grocery)

grocery_df.rename(columns = {'value':'Per_decrease_grocery'}, inplace = True)

# transit

transit = location_df[3]

transit_df = pd.DataFrame(transit)

transit_df.rename(columns = {'value':'Per_decrease_transit'}, inplace = True)


# Work place

workplace = location_df[4]

workplace_df = pd.DataFrame(workplace)

workplace_df.rename(columns = {'value':'Per_decrease_workplace'}, inplace = True)

# Residential

residential = location_df[5]

residential_df = pd.DataFrame(residential)

residential_df.rename(columns = {'value':'Per_decrease_residential'}, inplace = True)

In [10]:
# Create the final Mobility data
mobility_df = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(retail_df, parks_df, on = 'date', how = 'outer'),
        grocery_df, on = 'date', how = 'outer'),transit_df, on = 'date', how = 'outer'),
        workplace_df, on = 'date', how = 'outer'),residential_df, on = 'date', how = 'outer')

mobility_df.sort_values('date').head()

,date,Per_decrease_ret_rec,Per_decrease_parks,Per_decrease_grocery,Per_decrease_transit,Per_decrease_workplace,Per_decrease_residential
49,2020-02-16,6,28,0,-9,-23,5
48,2020-02-17,1,7,0,1,-2,1
47,2020-02-18,2,8,0,1,1,0
46,2020-02-19,1,5,0,0,0,1
45,2020-02-20,2,4,-2,1,0,0


In [11]:
# Merge cases data with mobility

combined_df = pd.merge(mobility_df, us_covid_df, on = 'date', how = 'left')

combined_df = combined_df.sort_values('date')

combined_df = combined_df[['date','cases', 'deaths', 'Per_decrease_ret_rec', 'Per_decrease_parks', 'Per_decrease_grocery',
                           'Per_decrease_transit', 'Per_decrease_workplace', 'Per_decrease_residential']]

In [12]:
combined_df.head()

,date,cases,deaths,Per_decrease_ret_rec,Per_decrease_parks,Per_decrease_grocery,Per_decrease_transit,Per_decrease_workplace,Per_decrease_residential
49,2020-02-16,15,0,6,28,0,-9,-23,5
48,2020-02-17,25,0,1,7,0,1,-2,1
47,2020-02-18,25,0,2,8,0,1,1,0
46,2020-02-19,25,0,1,5,0,0,0,1
45,2020-02-20,27,0,2,4,-2,1,0,0


In [13]:
# Create a data frame with State and it's abbrevation

us_state_abbrev_df = pd.DataFrame(us_state_abbrev.items(), columns=['State', 'Abrevation'])

us_state_abbrev_list = us_state_abbrev.values()

In [54]:
list = ['AK']

locations = ['retailAndRecreation', 'parks', 'groceryAndPharmacy', 'transitStations','workplaces', 'residential']

location_state_df = []

for state in list:
    
    try:
    
        url_s = (f"https://pastelsky.github.io/covid-19-mobility-tracker/output/US/{state}/mobility.json")
        
        result = requests.get(url_s).json()
        
        print(f"Mobility data found for {state}")
        
        for place in locations:
        
            result = response['country'][place]['points']
            
            #location_state_df.append(state)
            
            #location_state_df.append(place)
            
            location_state_df.append(result)
        
    except:
        
        print(f"Mobility data not found for {state}")

    

Mobility data found for AK


In [55]:
#location_state_df{'AK':{'retail': [{date, value},{date, value}]}}

location_state_df[0]

[{'date': '2020-04-05', 'value': -49},
 {'date': '2020-04-04', 'value': -48},
 {'date': '2020-04-03', 'value': -42},
 {'date': '2020-04-02', 'value': -40},
 {'date': '2020-04-01', 'value': -38},
 {'date': '2020-03-31', 'value': -40},
 {'date': '2020-03-30', 'value': -40},
 {'date': '2020-03-29', 'value': -47},
 {'date': '2020-03-28', 'value': -48},
 {'date': '2020-03-27', 'value': -42},
 {'date': '2020-03-26', 'value': -40},
 {'date': '2020-03-25', 'value': -40},
 {'date': '2020-03-24', 'value': -39},
 {'date': '2020-03-23', 'value': -38},
 {'date': '2020-03-22', 'value': -43},
 {'date': '2020-03-21', 'value': -40},
 {'date': '2020-03-20', 'value': -33},
 {'date': '2020-03-19', 'value': -29},
 {'date': '2020-03-18', 'value': -25},
 {'date': '2020-03-17', 'value': -20},
 {'date': '2020-03-16', 'value': -11},
 {'date': '2020-03-15', 'value': -9},
 {'date': '2020-03-14', 'value': -7},
 {'date': '2020-03-13', 'value': -1},
 {'date': '2020-03-12', 'value': 4},
 {'date': '2020-03-11', 'value

In [39]:
a = dict(zip(list, retail))

In [40]:
# Retail
    
    retail_recreation = result['country']['retailAndRecreation'] ['points']
    
    result.append(retail_recreation)
    
    # Parks
    
    Parks = result['country']['parks'] ['points']
    
    result.append(Parks)
    
    # Grocery & Pharmacy
    
    grocery_pharmacy = result['country']['groceryAndPharmacy'] ['points']
    
    result.append(grocery_pharmacy)
    

[{'date': '2020-04-05', 'value': -49},
 {'date': '2020-04-04', 'value': -48},
 {'date': '2020-04-03', 'value': -42},
 {'date': '2020-04-02', 'value': -40},
 {'date': '2020-04-01', 'value': -38},
 {'date': '2020-03-31', 'value': -40},
 {'date': '2020-03-30', 'value': -40},
 {'date': '2020-03-29', 'value': -47},
 {'date': '2020-03-28', 'value': -48},
 {'date': '2020-03-27', 'value': -42},
 {'date': '2020-03-26', 'value': -40},
 {'date': '2020-03-25', 'value': -40},
 {'date': '2020-03-24', 'value': -39},
 {'date': '2020-03-23', 'value': -38},
 {'date': '2020-03-22', 'value': -43},
 {'date': '2020-03-21', 'value': -40},
 {'date': '2020-03-20', 'value': -33},
 {'date': '2020-03-19', 'value': -29},
 {'date': '2020-03-18', 'value': -25},
 {'date': '2020-03-17', 'value': -20},
 {'date': '2020-03-16', 'value': -11},
 {'date': '2020-03-15', 'value': -9},
 {'date': '2020-03-14', 'value': -7},
 {'date': '2020-03-13', 'value': -1},
 {'date': '2020-03-12', 'value': 4},
 {'date': '2020-03-11', 'value

In [ ]:
retail = []

parks = []

grocery = []

transit = []

workplaces = []

residential = []
